# Transport Data Science Project

## Introduction
### Transport Data Science Project (TDSP)
The Transport Data Science Project was a 4 week data science and research program centered on analyzing the New York City Department of Transportation's [Motor Vehicle Collision Dataset](https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Vehicles/bm4k-52h4/about_data). During my cycle of the TDSP program there were a roughly a thousand participants, I was one of 20 to be selected to present my research to the NYC Department of Transportation in a virtual research symposium. 

### This Notebook
Below is my work doing exploratory data analysis to understand the data better. The first few lines of code were problem sets from the program however quickly after that I do my own more complicated analysis to identify potential feature variables for different machine learning models. 

### ML Task: Predicting Injury vs Fatality for Collisions
In this project my main goal is to try to predict whether or not certain motor vehicle collision leads to at least one injury or at least one fatality. 

## Exploratory Data Analysis

### Loading Data